In [1]:
import ctypes
from ctypes import c_void_p

# Load the Rust compiled shared library
lib = ctypes.CDLL('/Users/victorh/Desktop/anoncreds/revocation_manager_ffi/agora-allosaurus-rs/target/release/libagora_allosaurus_rs.dylib')
lib.allosaurus_new_server.restype = c_void_p
server = lib.allosaurus_new_server()

if server:
    print("Server created successfully.")
else:
    print("Failed to create server.")


Server created successfully.
